In [ ]:
from simulation import evaluate, interpolate
import numpy as np

af = np.loadtxt('baseline.dat')
af = interpolate(af, 256, 3)
# perf, CD = evaluate(af, 0.67)

In [ ]:
perf_BL = 35.981436463391404
CD_BL = 0.004539919085800648

In [ ]:
airfoils = []
count = 0
for i in range(20):
    num = str(i+count).zfill(3)
    af = np.loadtxt('samples/airfoil'+num+'.dat')
    af = interpolate(af, 256, 3)
    perf, CD = evaluate(af, 0.67)
    if (perf > perf_BL) and (CD < CD_BL):
        airfoils.append(i+count)
airfoils

In [ ]:
airfoils